In [1]:
%load_ext autoreload
%autoreload 2

from commander.network import NetworkManager
from commander.network.protocol import (
    PingPacket,
    SetPositionPacket,
    SetVelocityPacket,
    FindLimitsPacket,
    ObservationPacket,
    RequestDebugInfoPacket,
    CheckLimitPacket,
    DoJigglePacket,
    DebugPacket,
    InfoPacket,
    ErrorPacket,
    ExperimentStartPacket,
    ExperimentStopPacket
)

from commander.log import setup_logging

from commander.network.constants import SetOperation, CartID
from time import time, sleep
import random
import numpy as np

setup_logging(file=False)

nm = NetworkManager(port="COM3")
nm.open()

initialised = False
while not initialised:
    try:
        nm.read_initial_output()
    except UnicodeDecodeError:
        pass
    else:
        initialised = True

root                          : DEBUG    Sat up logging...


#$  Received unkn~   === Cartpole Controller ===
#   Free memory: 81799
#   STEPPER_CURRENT: 1000
#   STEPPER_MICROSTEPS: 8
#(   STEPPER_DISTANCE_PER_ROTATION: 40.840694
#   track_length_steps: 0
#   track_length_distance: 0.000000
#   experiment_done: 0
   run_safely: 0
   has_failed: 0
#   limit_finding_mode: DONE
#   limit_check_mode: DONE
#   Setting up I2C
#   Configuring buttons
#   Setting up steppers.
#   Starting stepper library.
#   Initiating astepper 1
#   Setting up stepper drivers.
#   Setting up stepper driver 1.
CS (Vsense=1): 28
Microstepping: 32
Step delay in micros: 0
Sending 3
Received 101
Sending 9C7D7
Received 80
Sending A0000
Received 80
Sending C001C
Received 80
Sending E0050
Received 80
Sending E0080
Received 80
Sending C001F
Received 80
Sending A8202
Received 80
Sending 901B4
Received 80
Sending 3
Received C0
INFO: Motor is standing still.
Stall Guard value:0
Sending 3
Received C0
ERROR: Channel B seems to be unconnected!
INFO: Motor is stan

In [6]:
print(nm.serial.read_all().decode("ASCII", errors="ignore"))

expected index



In [2]:
nm.dump_packets()

# PingPacket

In [2]:
ping_pkt = PingPacket(timestamp=10)
nm.send_packet(ping_pkt)

pkt = nm.read_packet()
print(pkt)

commander.network.network     : DEBUG    Read packet: <PongPacket: id_: b'P', read_time: 2021-08-20 17:48:56.112302, timestamp: 10>


<PongPacket: id_: b'P', read_time: 2021-08-20 17:48:56.112302, timestamp: 10>


# UnknownPacket

In [3]:
nm.serial.write(b"a")
pkt = nm.read_packet()
print(pkt)

commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: Received unknown packet with ID: 97, read_time: 2021-08-20 15:59:08.108702>


<DebugPacket: id_: b'#', msg: Received unknown packet with ID: 97, read_time: 2021-08-20 15:59:08.108702>


# RequestDebugInfoPacket

In [3]:
request_dbg_pkt = RequestDebugInfoPacket()
nm.send_packet(request_dbg_pkt)

sleep(0.010)
nm.dump_packets()

commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: Free memory: 81687, read_time: 2021-08-20 17:48:59.664554>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: STEPPER_CURRENT: 1000, read_time: 2021-08-20 17:48:59.665553>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: STEPPER_MICROSTEPS: 8, read_time: 2021-08-20 17:48:59.666554>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: STEPPER_DISTANCE_PER_ROTATION: 40.840694, read_time: 2021-08-20 17:48:59.667554>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: track_length_steps: 0, read_time: 2021-08-20 17:48:59.668555>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: track_length_distance: 0.000000, read_time: 2021-08-20 17:48:59.672553>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: experiment_don

<DebugPacket: id_: b'#', msg: Free memory: 81687, read_time: 2021-08-20 17:48:59.664554>
<DebugPacket: id_: b'#', msg: STEPPER_CURRENT: 1000, read_time: 2021-08-20 17:48:59.665553>
<DebugPacket: id_: b'#', msg: STEPPER_MICROSTEPS: 8, read_time: 2021-08-20 17:48:59.666554>
<DebugPacket: id_: b'#', msg: STEPPER_DISTANCE_PER_ROTATION: 40.840694, read_time: 2021-08-20 17:48:59.667554>
<DebugPacket: id_: b'#', msg: track_length_steps: 0, read_time: 2021-08-20 17:48:59.668555>
<DebugPacket: id_: b'#', msg: track_length_distance: 0.000000, read_time: 2021-08-20 17:48:59.672553>
<DebugPacket: id_: b'#', msg: experiment_done: 0, read_time: 2021-08-20 17:48:59.676553>
<DebugPacket: id_: b'#', msg: run_safely: 0, read_time: 2021-08-20 17:48:59.680553>
<DebugPacket: id_: b'#', msg: has_failed: 0, read_time: 2021-08-20 17:48:59.681556>
<DebugPacket: id_: b'#', msg: limit_finding_mode: DONE, read_time: 2021-08-20 17:48:59.684553>
<DebugPacket: id_: b'#', msg: limit_check_mode: DONE, read_time: 2021-

# FindLimitsPacket

In [5]:
PKT = FindLimitsPacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: track_length_distance: 0.000000, read_time: 2021-08-20 17:49:40.531563>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: experiment_done: 0, read_time: 2021-08-20 17:49:40.532563>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: run_safely: 0, read_time: 2021-08-20 17:49:40.534564>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: has_failed: 0, read_time: 2021-08-20 17:49:40.535565>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: limit_finding_mode: DONE, read_time: 2021-08-20 17:49:40.536566>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: limit_check_mode: DONE, read_time: 2021-08-20 17:49:40.537562>
commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [fast], read_time:

<DebugPacket: id_: b'#', msg: track_length_distance: 0.000000, read_time: 2021-08-20 17:49:40.531563>
<DebugPacket: id_: b'#', msg: experiment_done: 0, read_time: 2021-08-20 17:49:40.532563>
<DebugPacket: id_: b'#', msg: run_safely: 0, read_time: 2021-08-20 17:49:40.534564>
<DebugPacket: id_: b'#', msg: has_failed: 0, read_time: 2021-08-20 17:49:40.535565>
<DebugPacket: id_: b'#', msg: limit_finding_mode: DONE, read_time: 2021-08-20 17:49:40.536566>
<DebugPacket: id_: b'#', msg: limit_check_mode: DONE, read_time: 2021-08-20 17:49:40.537562>
Sent packet: <FindLimitsPacket: id_: b'|', read_time: None>
<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [fast], read_time: 2021-08-20 17:49:40.677563>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT RETRACTED, read_time: 2021-08-20 17:49:41.050563>


<InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT RETRACTED, read_time: 2021-08-20 17:49:41.050563>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT HIT [slow], read_time: 2021-08-20 17:49:42.713560>
commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitFinder: LEFT LIMIT SET, read_time: 2021-08-20 17:49:42.721561>


...

<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT RETRACTED, read_time: 2021-08-20 17:49:55.026708>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT HIT [slow], read_time: 2021-08-20 17:49:56.763399>


<InfoPacket: id_: b'~', msg: LimitFinder: RIGHT LIMIT HIT [slow], read_time: 2021-08-20 17:49:56.763399>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitFinder: NOW DONE, read_time: 2021-08-20 17:49:57.918393>
commander.network.network     : DEBUG    Read packet: <FindLimitsPacket: id_: b'|', read_time: 2021-08-20 17:49:57.923370>


<InfoPacket: id_: b'~', msg: LimitFinder: NOW DONE, read_time: 2021-08-20 17:49:57.918393>
<FindLimitsPacket: id_: b'|', read_time: 2021-08-20 17:49:57.923370>


# CheckLimitPacket

In [6]:
PKT = CheckLimitPacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

Sent packet: <CheckLimitPacket: id_: b'/', read_time: None>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT HIT [fast], read_time: 2021-08-20 17:50:17.883101>


<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT HIT [fast], read_time: 2021-08-20 17:50:17.883101>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT RETRACTED, read_time: 2021-08-20 17:50:18.333023>


<InfoPacket: id_: b'~', msg: LimitChecker: LEFT LIMIT RETRACTED, read_time: 2021-08-20 17:50:18.333023>


...

<DebugPacket: id_: b'#', msg: B, read_time: 2021-08-20 17:50:21.491025>


commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: C, read_time: 2021-08-20 17:50:22.491023>


<DebugPacket: id_: b'#', msg: C, read_time: 2021-08-20 17:50:22.491023>


commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: LimitChecker: NOW DONE, read_time: 2021-08-20 17:50:24.908151>
commander.network.network     : DEBUG    Read packet: <CheckLimitPacket: id_: b'/', read_time: 2021-08-20 17:50:24.913077>
commander.network.network     : DEBUG    Read packet: <DebugPacket: id_: b'#', msg: Free memory: 81687, read_time: 2021-08-20 17:50:24.914041>


<InfoPacket: id_: b'~', msg: LimitChecker: NOW DONE, read_time: 2021-08-20 17:50:24.908151>
<DebugPacket: id_: b'#', msg: Free memory: 81687, read_time: 2021-08-20 17:50:24.914041>
<CheckLimitPacket: id_: b'/', read_time: 2021-08-20 17:50:24.913077>


# DoJigglePacket

In [5]:
PKT = DoJigglePacket
nm.dump_packets()

pkt = PKT()
nm.send_packet(pkt)
print(f"Sent packet: {pkt}")

response_pkt = nm.get_packet(PKT, block=True, callback=nm.printer_callback(excepts=(PKT, )))
nm.dump_packets()
print(response_pkt)

commander.network.network     : DEBUG    Read packet: <InfoPacket: id_: b'~', msg: Stopped steppers., read_time: 2021-08-20 17:10:38.401963>
commander.network.network     : DEBUG    Read packet: <SoftLimitReachedPacket: id_: b'\\', read_time: 2021-08-20 17:10:38.403878>
commander.network.network     : DEBUG    Read packet: <ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2021-08-20 17:10:38.404886, specifier: 2, value: -1>


<InfoPacket: id_: b'~', msg: Stopped steppers., read_time: 2021-08-20 17:10:38.401963>
<SoftLimitReachedPacket: id_: b'\\', read_time: 2021-08-20 17:10:38.403878>
<ExperimentInfoPacket: cart_id: 1, id_: b':', read_time: 2021-08-20 17:10:38.404886, specifier: 2, value: -1>
Sent packet: <DoJigglePacket: id_: b'\xa7', read_time: None>


commander.network.network     : DEBUG    Read packet: <DoJigglePacket: id_: b'\xa7', read_time: 2021-08-20 17:10:41.203342>


<DoJigglePacket: id_: b'\xa7', read_time: 2021-08-20 17:10:41.203342>


# SetPositionPacket

In [3]:
nm.dump_packets()

pkt = SetPositionPacket(operation=SetOperation.EQUAL, cart_id=CartID.ONE, value=500)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")

nm.dump_packets()

Sent packet: <SetPositionPacket: cart_id: 1, id_: b'x', operation: =, value: 500>


# SetVelocityPacket

In [ ]:
nm.dump_packets()

pkt = SetVelocityPacket(operation=SetOperation.ADD, cart_id=CartID.ONE, value=100)
nm.send_packet(pkt)

print(f"Sent packet: {pkt}")
sleep(1)

nm.dump_packets()

In [4]:
nm.close()